In [2]:
import pandas as pd
from datasets import load_from_disk

In [9]:
subset_name = "processed_Meta-Llama-3.1-8B-Instruct-evals__mbpp__details_results"

dataset = load_from_disk(f"data/{subset_name}")
df = dataset.to_pandas()

In [12]:
df.head()

,task_type,task_name,subtask_name,input_question,input_choice_list,input_final_prompts,input_correct_responses,output_prediction_text,output_parsed_answer,output_choice_completions,...,input_question_hash,input_final_prompts_hash,benchmark_label,eval_config,context,query,final_output_layer_32,final_output_layer_orig,extracted_functions_layer_32,extracted_functions_layer_orig
0,Generative,mbpp_chat,None,Write a function to check if given tuple is di...,None,[<|start_header_id|>user<|end_header_id|>\n\nY...,None,[\ndef check_distinct(tup):\n return len(tup)...,\ndef check_distinct(tup):\n return len(tup) ...,None,...,5f95e606207366d0b090735637ee25085e3a99edfc4272...,[9c7412dfa641dda248586db1638287547113760c3daa7...,MBPP,"{'max_gen_len': '256', 'max_prompt_len': '4500...",[<|start_header_id|>user<|end_header_id|>\n\nY...,[<|start_header_id|>user<|end_header_id|>\n\nY...,"\ndef check_distinct(tup):\n """"""\n This ...","\ndef check_distinct(tup):\n """"""\n This ...",def check_distinct(tup):\n return len(tup) ...,def check_distinct(tup):\n return len(tup) ...


In [ ]:
import json
import os
import re

from collections import defaultdict

layer_indices = [1, 10, 23, 'orig']

def sort_filenames(filenames):
    
    def extract_number(filename):
        # Extract the last number from the filename
        match = re.search(r'(\d+)\.json$', filename)
        return int(match.group(1)) if match else 0

    # Sort the filenames based on the extracted number
    return sorted(filenames, key=extract_number)

def find_last_checkpoints(directory, layer):
    files = []
    for filename in os.listdir(directory):
        if filename.endswith('.json'):
            if '_'+str(layer)+'_' in filename:
                files.append(filename)
    return sort_filenames(files)[-1]

                
def analyze_json_checkpoints(directory, filename):
    stats = defaultdict(int)
    all_keys = set()
    sample = None

    with open(directory, 'r') as file:
        try:
            data = json.load(file)
            stats['total_files'] += 1
            all_keys.update(data.keys())
            
            print(len(dict(dict(data.items())['results'].items())['extracted_functions_1']))
        
        except json.JSONDecodeError:
            stats['invalid_json_files'] += 1

    return stats, sample

# Usage
checkpoint_dir = 'checkpoints-old'
last_file = find_last_checkpoints(checkpoint_dir, 1)
stats, samplee = analyze_json_checkpoints(checkpoint_dir, last_file)

In [83]:
from typing import Any, Union

def print_json_keys(filename):
    all_keys = {}
    with open(filename, 'r') as file:
        try:
            data = json.load(file)
            print(data.keys())
            k1 = dict(data.items())
            print(k1['results'].keys())
            results = dict(k1['results'])
            for i, k in (results.keys()):
                print(results[k][0])
                    
        except json.JSONDecodeError:
            stats['invalid_json_files'] += 1

    return stats
    
path = f"checkpoints-old/{last_file}"
print_json_keys(path)


dict_keys(['layer_idx', 'instance_index', 'results'])
dict_keys(['instance_id', 'query', 'context', 'test_cases', 'final_output_1', 'extracted_functions_1', 'trajectories_and_context_1', 'context_expansion_1'])
0
<|start_header_id|>user<|end_header_id|>

You are an expert Python programmer, and here is your task:
Write a function to check if given tuple is distinct or not.
Your code should pass the following tests:
assert check_distinct((1, 4, 5, 6, 1, 4)) == False
assert check_distinct((1, 4, 5, 6)) == True
assert check_distinct((2, 3, 4, 5, 6)) == True<|eot_id|><|start_header_id|>assistant<|end_header_id|>

```python
<|start_header_id|>user<|end_header_id|>

You are an expert Python programmer, and here is your task:
Write a function to find the similar elements from the given two tuple lists.
Your code should pass the following tests:
assert similar_elements((3, 4, 5, 6),(5, 7, 4, 10)) == (4, 5)
assert similar_elements((1, 2, 3, 4),(5, 4, 3, 7)) == (3, 4)
assert similar_elements((11

defaultdict(int, {'total_files': 3})